# Step 03: Initiate Post-Processing

This notebook writes cycle summary data to the Risk_Modeler_PremiumIQ_Variable table in SQL Server.

**Data Written:**
- `DB_{peril_code}`: Full EDM database name (with random suffix) for each EDM
- `srcedm_{peril_code}`: Raw EDM name (from config) for each EDM
- `products_and_perils`: JSON of Products and Perils configuration
- `current_date_value`: Current date value from configuration (e.g., "202503")
- `DB_USAP`: Full RDM database name from Moody's API
- `Status`: NULL (initial status for new rows)

**Overwrite Rules:**
- If no existing rows for the InforceDate, proceeds with insert
- If existing rows have `Status` = NULL or 'FINISHED', deletes and overwrites
- If any existing row has a different `Status` value, stops processing and displays an error

**Prerequisites:**
- Configuration must be validated (VALID or ACTIVE status)
- RDM export must be complete (Stage 06, Step 01-02)

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import logging
logging.getLogger('helpers.sqlserver').setLevel(logging.ERROR)

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.database import execute_query
from helpers.configuration import read_configuration
from helpers.irp_integration import IRPClient
from helpers.post_processing import build_post_processing_rows, write_post_processing_data, check_can_overwrite

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_03_Initiate_Post_Processing.ipynb')

# Display context
ux.header("Initiate Post-Processing")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"Step tracking initialized for '{context.step_name}'")

## 2) Retrieve Configuration Data

In [ ]:
# Get cycle ID
ux.subheader("Retrieve Configuration")

cycle_result = execute_query(
    "SELECT id FROM irp_cycle WHERE cycle_name = %s",
    (context.cycle_name,)
)

if cycle_result.empty:
    raise ValueError(f"Cycle not found: {context.cycle_name}")

cycle_id = int(cycle_result.iloc[0]['id'])

# Get configuration for this cycle
config_result = execute_query(
    "SELECT id, status, created_ts FROM irp_configuration WHERE cycle_id = %s ORDER BY created_ts DESC LIMIT 1",
    (cycle_id,)
)

if config_result.empty:
    ux.error("No configuration found for this cycle")
    ux.info("Please complete Step 02: Validate Configuration File first")
    raise ValueError("No configuration found for cycle")

config_id = int(config_result.iloc[0]['id'])
config_status = config_result.iloc[0]['status']
config_created = config_result.iloc[0]['created_ts']

# Verify status is VALID or ACTIVE
if config_status not in ['VALID', 'ACTIVE']:
    ux.error(f"Configuration status is '{config_status}' (expected VALID or ACTIVE)")
    raise ValueError(f"Configuration must be VALID or ACTIVE, found: {config_status}")

# Display configuration summary
config_info = [
    ["Configuration ID", config_id],
    ["Status", config_status],
    ["Created", config_created.strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(config_info, headers=["Property", "Value"])
ux.success("Configuration verified")

# Read configuration data
config_data = read_configuration(config_id)
configuration_data = config_data.get('configuration_data', {})

# Extract key fields for display
metadata = configuration_data.get('Metadata', {})
databases = configuration_data.get('Databases', [])
products_perils = configuration_data.get('Products and Perils', [])

current_date_value = metadata.get('Current Date Value')
rdm_raw_name = metadata.get('Export RDM Name')
edm_count = len(databases)

ux.info(f"\nCurrent Date Value: {current_date_value}")
ux.info(f"Export RDM Name: {rdm_raw_name}")
ux.info(f"Number of EDMs: {edm_count}")
ux.info(f"Products and Perils entries: {len(products_perils)}")

step.log(f"Configuration verified: ID={config_id}, Status={config_status}")

## 3) Build Post-Processing Data

In [ ]:
# Initialize IRP client and build rows
ux.subheader("Build Post-Processing Rows")

irp_client = IRPClient()

ux.info("Resolving EDM and RDM full names from Moody's API...")
rows = build_post_processing_rows(configuration_data, irp_client)

ux.success(f"Built {len(rows)} rows for insertion")
step.log(f"Built {len(rows)} post-processing rows")

## 4) Preview Data

In [ ]:
# Preview data to be written
ux.subheader("Data Preview")

# Build preview table
preview_data = []
for row in rows:
    # Truncate long values for display
    value = row['variable_value']
    if len(value) > 60:
        value = value[:57] + '...'
    preview_data.append([row['inforce_date'], row['variable_name'], value])

ux.table(preview_data, headers=["InforceDate", "VariableName", "VariableValue"])

ux.info(f"\nTarget table: DW_EXP_MGMT_USER.Risk_Modeler_PremiumIQ_Variable")
ux.info(f"Connection: ASSURANT")

## 5) Check Existing Data

In [ ]:
# Check for existing data and verify we can overwrite
ux.subheader("Check Existing Data")

inforce_date = rows[0]['inforce_date'] if rows else None

if not inforce_date:
    ux.error("No rows to write - cannot determine InforceDate")
    raise ValueError("No rows built for post-processing")

ux.info(f"Checking for existing rows with InforceDate={inforce_date}...")

check_result = check_can_overwrite(inforce_date)

if not check_result['exists']:
    ux.success("No existing rows found - safe to proceed with insert")
    can_proceed = True
elif check_result['can_overwrite']:
    ux.info(f"Found {len(check_result['rows'])} existing row(s)")
    # Show existing rows
    existing_table = [
        [row['VariableName'], row['Status'] or 'NULL']
        for row in check_result['rows']
    ]
    ux.table(existing_table, headers=["VariableName", "Status"])
    ux.success("All existing rows have Status=NULL or FINISHED - safe to overwrite")
    can_proceed = True
else:
    ux.error(f"Found {len(check_result['blocking_rows'])} row(s) with blocking status")
    # Show blocking rows
    blocking_table = [
        [row['VariableName'], row['Status']]
        for row in check_result['blocking_rows']
    ]
    ux.table(blocking_table, headers=["VariableName", "Status"])
    ux.error("Cannot overwrite rows that are currently being processed")
    ux.info("Wait for processing to complete (Status=FINISHED) before re-running this step")
    can_proceed = False

step.log(f"Existing data check: exists={check_result['exists']}, can_overwrite={check_result['can_overwrite']}")

## 6) Write to Database

In [ ]:
# Write data to SQL Server
ux.subheader("Write to Database")

if not can_proceed:
    ux.error("Cannot proceed - existing rows have blocking status")
    ux.info("See above for details on blocking rows")
    raise ValueError("Post-processing blocked by existing rows with non-overwritable status")

ux.info(f"Deleting existing rows for InforceDate={inforce_date}...")
ux.info(f"Inserting {len(rows)} rows into Risk_Modeler_PremiumIQ_Variable with Status=NULL...")

try:
    inserted_count = write_post_processing_data(rows)
    ux.success(f"Successfully inserted {inserted_count} rows")
    step.log(f"Inserted {inserted_count} rows to Risk_Modeler_PremiumIQ_Variable for InforceDate={inforce_date}")
    write_success = True
except Exception as e:
    ux.error(f"Failed to write data: {str(e)}")
    step.log(f"Write failed: {str(e)}")
    write_success = False
    raise

## 7) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

# Prepare output data
output_data = {
    'rows_inserted': inserted_count,
    'edm_count': edm_count,
    'current_date_value': current_date_value,
    'rdm_raw_name': rdm_raw_name,
    'target_table': 'Risk_Modeler_PremiumIQ_Variable',
    'connection': 'ASSURANT',
    'database': 'DW_EXP_MGMT_USER'
}

# Complete the step
step.complete(output_data)

ux.success("\n" + "="*60)
ux.success("POST-PROCESSING COMPLETED SUCCESSFULLY")
ux.success("="*60)

ux.info(f"\nRows inserted: {inserted_count}")
ux.info(f"EDMs processed: {edm_count}")
ux.info(f"InforceDate: {rows[0]['inforce_date'] if rows else 'N/A'}")
ux.info(f"Target: ASSURANT.DW_EXP_MGMT_USER.Risk_Modeler_PremiumIQ_Variable")